In [1]:
from keras.applications.resnet50 import preprocess_input, decode_predictions,ResNet50
from keras.preprocessing import image
import numpy as np
from matplotlib import pyplot as plt 


Using TensorFlow backend.


In [2]:
from keras import backend as K
import tensorflow as tf

#烤面包机
object_type_to_fake = 859

INPUT_SHAPE=[1,224,224,3]
OUTPUT_DIM=1000

session=tf.Session()


#x = K.placeholder(name='x', dtype='float32', shape=INPUT_SHAPE)
y = K.placeholder(name='y', dtype='int32')

# Optimize directly on noize (perturbation)
#perturbation = tf.get_variable(name="perturbation", shape=INPUT_SHAPE, dtype=tf.float32, initializer=tf.zeros_initializer)

#x_perturbation = x + perturbation

adv = tf.get_variable(name="adv", shape=INPUT_SHAPE, dtype=tf.float32, initializer=tf.zeros_initializer)


#使用基于imagenet2012训练的参数实例化ResNet50
model = ResNet50(input_tensor=adv,weights='imagenet')

logits = model.output

In [3]:
img = image.load_img("../picture/pig.jpg", target_size=(224, 224))
#plt.imshow(img) 
#plt.show() 

#print(img)
original_image = image.img_to_array(img)
original_image = np.expand_dims(original_image, axis=0)
original_image = preprocess_input(original_image)
#print(original_image)
#preds = model.predict(original_image)
#print('Predicted:', decode_predictions(preds, top=3)[0])

#label=np.argmax(preds)

#print("label={}".format(label))

#predictions=decode_predictions(preds, top=3)[0]
#print(predictions)

#print(logits)
session.run(tf.global_variables_initializer())
#session.run(tf.assign(perturbation,tf.zeros(INPUT_SHAPE)))

#print(original_image)

predictions=session.run([logits],{adv:original_image,K.learning_phase():False})

#print(predictions)

predictions = np.squeeze(predictions)
label=np.argmax(predictions)
print("label={}".format(label))

label=772


In [ ]:
#迭代求解
epochs=500
lr=0.1
target_label=288


#优化器调整的参数
var_list = [adv]
#损失函数/目标函数
#f=K.categorical_crossentropy(y, logits, from_logits=False)[0]
f = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=[y])
optimizer = tf.train.AdamOptimizer(
                learning_rate=lr, beta1=0.9, beta2=0.999, epsilon=1e-08)
train_step = optimizer.minimize(f, var_list=var_list)


# 初始化参数  非常重要 Adam的参数也需要这样初始化 GradientDescent可以省略这一步
session.run(tf.global_variables_initializer())

session.run(tf.assign(adv,original_image))

#K.learning_phase(): False 设置为预测模式
#To make use of the learning phase, simply pass the value "1" (training mode) or "0" (test mode) to feed_dict:
for epoch in range(epochs):
    
    loss,_,adv_img,predictions=session.run([f,train_step,adv,logits],
                                           {y:target_label,K.learning_phase():False})
    
    predictions = np.squeeze(predictions)
    label=np.argmax(predictions)
    
    print("epoch={} loss={} label={}".format(epoch,loss,label))
    
    #如果定向攻击成功
    if label == target_label:
        #preds = model.predict(adv_img)
        #print('Predicted:', decode_predictions(preds, top=3)[0])

        #predictions=decode_predictions(preds, top=3)[0]
        #print(predictions)
        break  


epoch=0 loss=[6.9094715] label=188
epoch=1 loss=[6.9094715] label=188
epoch=2 loss=[6.9094715] label=188
epoch=3 loss=[6.9094715] label=188
epoch=4 loss=[6.9094715] label=188
epoch=5 loss=[6.9094715] label=188
epoch=6 loss=[6.9094715] label=188
epoch=7 loss=[6.9094715] label=188
epoch=8 loss=[6.9094715] label=188
epoch=9 loss=[6.9094715] label=188
epoch=10 loss=[6.9094715] label=188
epoch=11 loss=[6.9094715] label=188
epoch=12 loss=[6.9094715] label=188
epoch=13 loss=[6.9094715] label=188
epoch=14 loss=[6.9094715] label=188
epoch=15 loss=[6.9094715] label=188
epoch=16 loss=[6.9094715] label=188
epoch=17 loss=[6.9094715] label=188
epoch=18 loss=[6.9094715] label=188
epoch=19 loss=[6.9094715] label=188
epoch=20 loss=[6.9094715] label=188
epoch=21 loss=[6.9094715] label=188
epoch=22 loss=[6.9094715] label=188
epoch=23 loss=[6.9094715] label=188
